In [36]:
!pip install transformers
!pip install sentencepiece
!pip install -U sentence-transformers
!pip install flask-ngrok

import numpy as np
import torch

import pandas as pd
import os
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers.optimization import  Adafactor 
import time
import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


For Combining Text

In [68]:
import urllib.request
import zipfile
url = 'https://gitlab.com/shimorina/webnlg-dataset/-/archive/master/webnlg-dataset-master.zip?path=release_v3.0/en/train'
urllib.request.urlretrieve(url, 'web.zip')
with zipfile.ZipFile('web.zip', 'r') as zip_ref:
    zip_ref.extractall('web')
import glob
import os
import re
import xml.etree.ElementTree as ET
import pandas as pd
files = glob.glob("/content/web/webnlg-dataset-master-release_v3.0-en-train/release_v3.0/en/train/**/*.xml", recursive=True)
triple_re=re.compile('(\d)triples')
data_dct={}
for file in files:
    tree = ET.parse(file)
    root = tree.getroot()
    triples_num=int(triple_re.findall(file)[0])
    for sub_root in root:
        for ss_root in sub_root:
            strutured_master=[]
            unstructured=[]
            for entry in ss_root:
                unstructured.append(entry.text)
                strutured=[triple.text for triple in entry]
                strutured_master.extend(strutured)
            unstructured=[i for i in unstructured if i.replace('\n','').strip()!='' ]
            strutured_master=strutured_master[-triples_num:]
            strutured_master_str=(' && ').join(strutured_master)
            data_dct[strutured_master_str]=unstructured
mdata_dct={"prefix":[], "input_text":[], "target_text":[]}
for st,unst in data_dct.items():
    for i in unst:
        mdata_dct['prefix'].append('webNLG')
        mdata_dct['input_text'].append(st)
        mdata_dct['target_text'].append(i)


df=pd.DataFrame(mdata_dct)
df.to_csv('webNLG2020_train.csv')

train_df=pd.read_csv('webNLG2020_train.csv', index_col=[0])

train_df=train_df.iloc[  :35000,:]
train_df=train_df.sample(frac = 1)
batch_size=8
num_of_batches=len(train_df)/batch_size
num_of_epochs=4
num_of_batches=int(num_of_batches)

if torch.cuda.is_available():
    dev = torch.device("cuda:0") 
    print("Running on the GPU")
else:
    dev = torch.device("cpu")
    print("Running on the CPU")

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)
#moving the model to device(GPU/CPU)
model.to(dev)


optimizer = Adafactor(
    model.parameters(),
    lr=1e-3,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False
)


from IPython.display import HTML, display

def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(loss=loss,value=value, max=max))
num_of_epochs=1

Running on the GPU


In [69]:
####################################################################
#Sets the module in training mode
model.train()

loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
  print('Running epoch: {}'.format(epoch))
  
  running_loss=0

  out = display(progress(1, num_of_batches+1), display_id=True)
  for i in range(num_of_batches):
    inputbatch=[]
    labelbatch=[]
    new_df=train_df[i*batch_size:i*batch_size+batch_size]
    for indx,row in new_df.iterrows():
      input = 'WebNLG: '+row['input_text']+'</s>' 
      labels = row['target_text']+'</s>'   
      inputbatch.append(input)
      labelbatch.append(labels)
    inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
    labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
    inputbatch=inputbatch.to(dev)
    labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
    optimizer.zero_grad()

    # Forward propogation
    outputs = model(input_ids=inputbatch, labels=labelbatch)
    loss = outputs.loss
    loss_num=loss.item()
    logits = outputs.logits
    running_loss+=loss_num
    if i%10 ==0:      
      loss_per_10_steps.append(loss_num)
    out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
    loss.backward()

    #updating the params
    optimizer.step()
    
  running_loss=running_loss/int(num_of_batches)
  print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))
#################################################################################


model.eval()
input_ids = tokenizer.encode("WebNLG: sidharth | hometown | Delhi && sidharth | play |  football </s>", return_tensors="pt")  # Batch size 1
input_ids=input_ids.to(dev)
outputs = model.generate(input_ids)
tokenizer.decode(outputs[0])

Running epoch: 1


Epoch: 1 , Running loss: 0.4988790895019259


'<pad> Sidharth plays football for the city of Delhi.</s>'

In [70]:
PATH = '/content/drive/MyDrive/Projects/NST/generative_model/model.pt' 
#torch.save(model, PATH)
model = torch.load(PATH)
model.eval()

For POS tags

In [39]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

For Similarity

In [42]:
from sentence_transformers import SentenceTransformer
from scipy import spatial
#sentences = ["That is a happy person", "That is a very happy person"]
file1 = open('sens.txt', 'r')
Lines = file1.readlines()
data = []
for line in Lines:
    if len(line)< 10:
      continue
    data.append(line.strip())

modelEm = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v1')
embeddings = modelEm.encode(data)
#print(embeddings)


In [43]:
embeddings.shape

(1944, 384)

Final Functions

In [160]:
def getSimilarity(embd1,embd2):
  return 1 - spatial.distance.cosine(embd1, embd2)

def findSimilarSen(input_sen):
  embd = modelEm.encode(input_sen)
  idx = 0
  max_similarity = 0
  
  for i,embedding in enumerate(embeddings):
    val = getSimilarity(embd,embedding)
    if val > max_similarity:
      max_similarity = val
      idx = i
  return data[i],max_similarity

def transform(input_sen):
  simSen,sim = findSimilarSen(input_sen)

  if sim < .5:
    return input_sen

  JJ = None
  isentence = word_tokenize(input_sen)
  ssentence = word_tokenize(simSen)

  
  for tup in nltk.pos_tag(ssentence):
    if tup[1][:2] == 'JJ':
      JJ = tup[0]

  if JJ is None:
    return input_sen
  
  taggedSen = []
  for tup in nltk.pos_tag(isentence):
    if tup[1][:2] == 'JJ':
      taggedSen.append(JJ)
    if tup[1][:2] == 'VB' or tup[1][:2] == 'DT':
      continue
    else:
      taggedSen.append(tup[0]) 
  #print('WebNLG: '+' | '.join(taggedSen)+'</s>')
  input_ids = tokenizer.encode('WebNLG: '+' | '.join(taggedSen)+'</s>', return_tensors="pt")  # Batch size 1
  input_ids=input_ids.to(dev)
  outputs = model.generate(input_ids)
  return tokenizer.decode(outputs[0])


Hosting

In [162]:
from flask import Flask, jsonify, request, render_template
from bs4 import BeautifulSoup
#from model import ChatBot
from model1 import *
from flask_ngrok import run_with_ngrok

import json 
import warnings
warnings.filterwarnings('ignore')
  

if __name__=='__main__':
    pass

    
# app
app = Flask(__name__)
run_with_ngrok(app) 
# routes

@app.route('/')
def home():
    return render_template('index.html')
    
@app.route('/', methods=['POST'])

def predict():
    # chatbot = ChatBot()
    data = request.get_json(force=True)
    print(data)
    #reply = chatbot.get_reply(data['message'])
    msg = transform(data['message'])
    msg = BeautifulSoup(msg, "lxml").text
    reply = generate_res(msg)

    #bodyFat = predictBodyFat(np.asarray(data['raw_img']).astype(np.uint8))
    print("Message: ",msg)
    print("Reply: ",reply)
    # send back to browser
    
    # return data 
    return jsonify(results=[reply,msg])


if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2c6f-34-86-134-116.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{'message': 'Hi, how are you ?'}


INFO:werkzeug:127.0.0.1 - - [07/Aug/2022 11:39:45] "POST / HTTP/1.1" 200 -


Message:  Hi, how are you ?
Reply:  I'm good, you?
{'message': 'The pizza was very bad'}


INFO:werkzeug:127.0.0.1 - - [07/Aug/2022 11:39:55] "POST / HTTP/1.1" 200 -


Message:   The pizza is a dish that is a bad food.
Reply:  I think you mean bad pizza.


In [159]:
#Example
transform(' The pizza is very bad')

WebNLG: pizza | very | next | bad</s>


'<pad> The pizza is a dish that is a bad food.</s>'